In [2]:
import random
from datetime import datetime
from helper import generate_random_date, generate_random_image, hash_password
from dictionary import *

In [3]:
def gen_enjoyer(id):
    first_name = random.choice(FIRST_NAMES)
    last_name = random.choice(LAST_NAMES)
    full_name = f"{first_name} {last_name}"
    username = first_name.lower() + last_name.lower()[0] + "{:04d}".format(random.randint(0, 9999))
    password = hash_password(username[::-1])
    email = username + random.choice(EMAIL_SUFFIX)
    bio = f"I love {random.choice(GENRES)} music."
    return [id, full_name, username, password, email, bio] # insert_query("Enjoyer", id, full_name, username, email, bio)

def gen_artist(id):
    first_name = random.choice(FIRST_NAMES)
    last_name = random.choice(LAST_NAMES)
    full_name = f"{first_name} {last_name}"
    username = first_name.lower() + last_name.lower()[0] + "{:04d}".format(random.randint(0, 9999))
    password = hash_password(username[::-1])
    email = username + random.choice(EMAIL_SUFFIX)
    bio = f"I'm a musician from {random.choice(LOCATIONS)}!"
    verified = random.choice([True, False])
    return [id, full_name, username, password, email, bio, verified] # print(insert_query("Artist", ))

def gen_album(id):
    name = f"{random.choice(ALBUM_1)} {random.choice(ALBUM_2)} {random.choice(ALBUM_3)}"
    creation_date = generate_random_date()
    cover_art = generate_random_image(256, random.randint(3,6))
    return [id, name, creation_date, cover_art]

def gen_playlist(id, enjoyer_id):
    name = f"{random.choice(PLAYLIST_1)} {random.choice(PLAYLIST_2)}"
    creation_date = generate_random_date()
    description = f"{random.choice(PLAYLIST_DESC_PREFIX)} {random.choice(PLAYLIST_DESC_SUFFIX)}"
    cover_art = generate_random_image(256, random.randint(3,6))
    return [id, enjoyer_id, name, creation_date, description, cover_art]

def gen_track(id, album_id):
    name = f"{random.choice(SONG_1)} {random.choice(SONG_2)} {random.choice(SONG_3)}"
    creation_date = generate_random_date()
    genre = random.choice(GENRES)
    length_seconds = random.randint(60,600)
    return [id, name, creation_date, album_id, genre, length_seconds]

def gen_playlist_tracks(playlist_id, track_id):
    return [playlist_id, track_id]

def gen_performs_in(artist_id, track_id, plays):
    return [artist_id, track_id, plays]

def gen_friend(friend_id1, friend_id2):
    since = generate_random_date(datetime(2000, 1, 1))
    return [friend_id1, friend_id2, since]

def gen_reaction(user_id, content_id, txt_type):
    p = random.random()
    if p<0.33:
        txt = f"{random.choice(TXT_1)} {random.choice(TXT_2)} {random.choice(txt_type)}"
        emoji = None
    elif p<0.66:
        txt = None
        emoji = random.choice(EMOJIS)
    else:
        txt = f"{random.choice(TXT_1)} {random.choice(TXT_2)} {random.choice(txt_type)}"
        emoji = random.choice(EMOJIS)
    return [user_id, content_id, txt, emoji]

def gen_follows(enjoyer_id, artist_id):
    return [enjoyer_id, artist_id]

def gen_saved(enjoyer_id, content_id):
    return [enjoyer_id, content_id]

In [4]:
N_ENJOYER = 1000
N_ARTIST = 100
N_ALBUM = 200
N_TRACKS_ALBUM = (5, 15)
N_PLAYLIST = 1000
N_TRACKS_PLAYLIST = (0, 40)
N_FRIENDS_PER_USER = (0,20)
N_REACTION_PER_USER  = (0,25)  
N_REACTION_PER_ARTIST  = (0,5) 
N_FOLLOWS_PER_USER = (0,15)
N_SAVED_PER_USER = (0,50)

enjoyers = [gen_enjoyer(i) for i in range(0,N_ENJOYER)]
artists = [gen_artist(i) for i in range(N_ENJOYER, N_ENJOYER+N_ARTIST)]
albums = [gen_album(i) for i in range(0,N_ALBUM)]
playlists = [gen_playlist(i, random.choice(enjoyers)[0]) for i in range(N_ALBUM, N_ALBUM+N_PLAYLIST)]

content_i = len(albums) + len(playlists)
tracks = []
performs_in = []
for album in albums:
    artist = random.choice(artists)
    instrument = random.choice(INSTRUMENTS)
    for i in range(0, random.randint(*N_TRACKS_ALBUM)):
        tracks.append(gen_track(content_i, album[0]))
        performs_in.append(gen_performs_in(artist[0], content_i, instrument))
        for i in range(0, random.randint(0,5)):
            if random.random() < 0.5:
                performs_in.append(gen_performs_in(random.choice(artists)[0], content_i, random.choice(INSTRUMENTS)))
        content_i += 1

playlist_tracks = []
for playlist in playlists:
    for i in range(0, random.randint(*N_TRACKS_PLAYLIST)):
        playlist_tracks.append(gen_playlist_tracks(playlist[0], random.choice(tracks)[0]))

friends = []
enjoyers_A = enjoyers[0:int(N_ENJOYER/2)]
enjoyers_B = enjoyers[int(N_ENJOYER/2):N_ENJOYER]
for a in enjoyers_A:
    for i in range(0, random.randint(*N_FRIENDS_PER_USER)):
        friends.append(gen_friend(a[0], random.choice(enjoyers_B)[0]))

reactions = []
for enjoyer in enjoyers:
    for i in range(0, random.randint(*N_REACTION_PER_USER)):
        p = random.random()
        if p<0.33:
            content_id = random.choice(albums)[0]
            txt_type = TXT_ALBUM
        elif p<0.66:
            content_id = random.choice(playlists)[0]
            txt_type = TXT_PLAYLIST
        else:
            content_id = random.choice(tracks)[0]
            txt_type = TXT_SONG
        reactions.append(gen_reaction(enjoyer[0], content_id, txt_type))

for artist in artists:
    for i in range(0, random.randint(*N_REACTION_PER_ARTIST)):
        p = random.random()
        if p<0.33:
            content_id = random.choice(albums)[0]
            txt_type = TXT_ALBUM
        elif p<0.66:
            content_id = random.choice(playlists)[0]
            txt_type = TXT_PLAYLIST
        else:
            content_id = random.choice(tracks)[0]
            txt_type = TXT_SONG
        reactions.append(gen_reaction(artist[0], content_id, txt_type))

follows = []
for enjoyer in enjoyers:
    for i in range(0, random.randint(*N_FOLLOWS_PER_USER)):
        follows.append(gen_follows(enjoyer[0], random.choice(artists)[0]))

saves = []
for enjoyer in enjoyers:
    for i in range(0, random.randint(*N_SAVED_PER_USER)):
        saves.append(gen_follows(enjoyer[0], random.choice(albums+tracks+playlists)[0]))

# User<-{Enjoyer, Artist} id ve verified dışında her şeyi User olarak INSERT'le. Enjoyer ve Artist ile ID üzerinden bağla.
# Content<-{Album, Playlist, Track} benzer işler.
# Imagelar düzgün oluyor mu? Encode decode işleriiii. :)))

In [11]:
output_dir = 'msn.sql'
objects = {"Enjoyer": enjoyers,
            "Artist": artists,
            "Album": albums,
            "Playlist": playlists,
            "Track": tracks,
            "PerformsIn": performs_in,
            "PlaylistTracks": playlist_tracks,
            "Friend": friends,
            "Reaction": reactions,
            "Follows": follows,
            "Saved": saves}

def insert_query(table: str, *params):
    params_str = ""
    for p in params:
        if p == None:
            params_str += "NULL"
        else:
            if isinstance(p, int):
                params_str += str(p)
            elif isinstance(p, (bytes, bytearray)):
                params_str += str(p)
            elif isinstance(p, str):
                params_str += f'"{str(p)}"'
            elif isinstance(p, bool):
                params_str += str(p)
        params_str += ","
    return f"INSERT INTO `{table}` VALUES ({params_str[:-1]});\n"

with open(output_dir, 'wb') as file:
    file.write("set autocommit=0;\n".encode())
    for (table, entries) in objects.items():
        for entry in entries:
            q = insert_query(table, *entry)
            file.write(q.encode())
    file.write("commit;".encode())

In [5]:
with open("test.txt", "wb") as file:
    file.write(playlists[5][5])